#### Simple Website Analyzer By URL (GEN AI APP) Using Langchain

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGSMITH_API_KEY"]=os.getenv("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")

In [3]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [6]:
loader=WebBaseLoader("https://dev.to/lovestaco/model-context-protocol-the-secret-sauce-behind-smart-ai-tools-5mc")
loader

In [7]:
docs=loader.load()
docs

[Document(metadata={'source': 'https://dev.to/lovestaco/model-context-protocol-the-secret-sauce-behind-smart-ai-tools-5mc', 'title': 'Model Context Protocol: The Secret Sauce Behind Smart AI Tools - DEV Community', 'description': 'Ever wondered how tools like Cursor manage to remember what you’re working on across different files,... Tagged with webdev, programming, beginners, ai.', 'language': 'en'}, page_content="\n\n\n\nModel Context Protocol: The Secret Sauce Behind Smart AI Tools - DEV Community\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\nNavigation menu\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n                Powered by\n                Search\n\n\n\n                Algolia\n            \n\n\n\n\n\n\n\n\n              Log in\n            \n\n\n            Create account\n          \n\n\n\n\n\n\n\nDEV Community\n\nClose\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n

In [8]:
### Load Data--> Docs-->Divide our Docuemnts into chunks dcouments-->text-->vectors-->Vector Embeddings--->Vector Store DB
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(docs)

In [9]:
documents

[Document(metadata={'source': 'https://dev.to/lovestaco/model-context-protocol-the-secret-sauce-behind-smart-ai-tools-5mc', 'title': 'Model Context Protocol: The Secret Sauce Behind Smart AI Tools - DEV Community', 'description': 'Ever wondered how tools like Cursor manage to remember what you’re working on across different files,... Tagged with webdev, programming, beginners, ai.', 'language': 'en'}, page_content='Model Context Protocol: The Secret Sauce Behind Smart AI Tools - DEV Community\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\nNavigation menu\n\n\n\n\n\n\n\n\n\n\n\n\n\nSearch\n\n\n\n\n                Powered by\n                Search\n\n\n\n                Algolia\n            \n\n\n\n\n\n\n\n\n              Log in\n            \n\n\n            Create account\n          \n\n\n\n\n\n\n\nDEV Community\n\nClose\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n

In [10]:
from langchain_openai import OpenAIEmbeddings
embeddings=OpenAIEmbeddings()

In [11]:
from langchain_community.vectorstores import FAISS
vectorstoredb=FAISS.from_documents(documents,embeddings)

In [12]:
vectorstoredb

In [13]:
## Query From a vector db
query="If your company is developing AI-powered tools, adopting or creating an MCP can be a game-changer. Here’s why:"
result=vectorstoredb.similarity_search(query)
result[0].page_content

'Why Use MCP?\n\n\n\nConsistent AI Behavior – Ensures AI models behave predictably across different environments.\n\nInteroperability – Lets different tools and editors work with the same AI model without compatibility issues.\n\nReduced Setup Hassle – Developers don’t need to rewrite integrations for every new platform.\n\nBetter Context Retention – Helps AI models maintain conversation history and user preferences across sessions.\n\n\n\n\n  Why an AI Product Company Should Build with MCP\n\nIf your company is developing AI-powered tools, adopting or creating an MCP can be a game-changer. Here’s why:'

In [14]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")

In [15]:
## Retrieval Chain, Document chain

from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

prompt=ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context:
<context>
{context}
</context>


"""
)

document_chain=create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
| ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000023D738A8DD0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000023D73273D10>, root_client=<openai.OpenAI object at 0x0000023D617FF210>, root_async_client=<openai.AsyncOpenAI object at 0x0000023D739AA290>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, confi

In [17]:
from langchain_core.documents import Document
document_chain.invoke({
    "input":"Why an AI Product Company Should Build with MCP",
    "context":[Document(page_content="Faster Integrations – Other platforms can plug into your AI model with minimal configuration.")]
})

"The answer to the question would depend on what the specific question is. However, based on the provided context, if you're asking about how other platforms interact with the AI model, the context suggests that other platforms can integrate with the AI model quickly and easily with minimal configuration required."

However, we want the documents to first come from the retriever we just set up. That way, we can use the retriever to dynamically select the most relevant documents and pass those in for a given question.

In [18]:
### Input--->Retriever--->vectorstoredb

vectorstoredb

In [19]:
retriever=vectorstoredb.as_retriever()
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)


In [20]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000023D71A80610>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context:\n<context>\n{context}\n</context>\n\n\n'), additional_kwargs={})])
            | 

In [21]:
## Get the response form the LLM
response=retrieval_chain.invoke({"input":"Why an AI Product Company Should Build with MCP"})
response['answer']

'Based on the provided context, an AI product company should build with MCP for several reasons:\n\n1. **Faster Integrations**: MCP allows other platforms to plug into the AI model with minimal configuration, reducing setup time.\n\n2. **Ecosystem Growth**: By encouraging third-party developers to build on top of the AI, MCP expands the product’s reach.\n\n3. **Improved User Experience**: MCP ensures smooth transitions when users switch between different tools, like VS Code and JetBrains IDE.\n\n4. **Competitive Edge**: A standardized protocol makes the AI more adaptable and future-proof, offering a competitive advantage.\n\nThe context emphasizes the plug-and-play nature of MCP, ensuring consistent AI behavior, interoperability, reduced setup hassle, and better context retention across different platforms and sessions.'

In [22]:

response

{'input': 'Why an AI Product Company Should Build with MCP',
 'context': [Document(id='533d5a56-9e7d-43f5-af8a-a9f6a59b493a', metadata={'source': 'https://dev.to/lovestaco/model-context-protocol-the-secret-sauce-behind-smart-ai-tools-5mc', 'title': 'Model Context Protocol: The Secret Sauce Behind Smart AI Tools - DEV Community', 'description': 'Ever wondered how tools like Cursor manage to remember what you’re working on across different files,... Tagged with webdev, programming, beginners, ai.', 'language': 'en'}, page_content='Why an AI Product Company Should Build with MCP\n\nIf your company is developing AI-powered tools, adopting or creating an MCP can be a game-changer. Here’s why:\n\nFaster Integrations – Other platforms can plug into your AI model with minimal configuration.\nEcosystem Growth – Encourages third-party developers to build on top of your AI, expanding your product’s reach.\nImproved User Experience – Ensures smooth transitions when users switch between different t

In [23]:
response['context']

[Document(id='533d5a56-9e7d-43f5-af8a-a9f6a59b493a', metadata={'source': 'https://dev.to/lovestaco/model-context-protocol-the-secret-sauce-behind-smart-ai-tools-5mc', 'title': 'Model Context Protocol: The Secret Sauce Behind Smart AI Tools - DEV Community', 'description': 'Ever wondered how tools like Cursor manage to remember what you’re working on across different files,... Tagged with webdev, programming, beginners, ai.', 'language': 'en'}, page_content='Why an AI Product Company Should Build with MCP\n\nIf your company is developing AI-powered tools, adopting or creating an MCP can be a game-changer. Here’s why:\n\nFaster Integrations – Other platforms can plug into your AI model with minimal configuration.\nEcosystem Growth – Encourages third-party developers to build on top of your AI, expanding your product’s reach.\nImproved User Experience – Ensures smooth transitions when users switch between different tools (e.g., VS Code → JetBrains IDE).\nCompetitive Edge – Having a standa